# Wikipedia Notable Life Expectancies 

## [Notebook 1 of 4:   Data Collection](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_collect_thanak_2022_06_10.ipynb)

## Context

This case study was chosen by this author as an end-to-end portfolio project for the following reasons:  
1. accessibility of the dataset
2. potential for use and demonstration of a wide range of skills, including coding in Python (using Pycharm and Jupyter Notebooks), version control (using Git, GitHub, and ReviewNB), web scraping (using Scrapy), relational database management (using SQLite and [SQLite Viewer](https://inloop.github.io/sqlite-viewer/)),  data cleaning (built-in string methods and regular expressions), Exploratory Data Analysis (EDA using numpy, pandas, matplotlib, plotly, and seaborn), data preprocessing, unsupervised learning (Kmeans and hierachical clustering), supervised learning (regression models), user interface for predictions.
3. in particular, a dataset with potential for regression modeling was chosen to gain more exposure to its associated algorithms as this author had relative broader experience with classification models in past projects.
    

## Objective

The...

## Data Dictionary
Variable: Description

## Data Collection
- Data was collected from 6/9/22 to 6/10/22, using Scrapy. 

### 6/9/2022

- The [Wikipedia](https://en.wikipedia.org/wiki/Main_Page) page, [List of Deaths by Year](https://en.wikipedia.org/wiki/Lists_of_deaths_by_year), contains entries for as early as 1987, to the present day.  
- 1994 was chosen as the start year for collection as it is the first year with entries following the current format: "Name, age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, cause of death (if known), and reference."
- For ease of pagination, [Deaths in January 1994](https://en.wikipedia.org/wiki/Deaths_in_January_1994) was the start url for scraping, proceeding month by month through subsequent pages.
- Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) scraped `month_year`, `day`, `name`, `info` ("age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, and cause of death (if known)"), and `link` for each entry on each month's page.  
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_deaths_94_to_22 within [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  This scraping was successful for January, 1994 through May, 2022 data.
- [Deaths in 2022 -- June](https://en.wikipedia.org/wiki/Deaths_in_2022#June)--the current month's page, at the time of scraping--varied in format, and was therefore scraped separately, after `num_references` was scraped for the previous entries.
- The original order of entries was preserved by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) in [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  Scrapy trades pagination order for speed, which is noticable when pagination is of higher magnitude.  Therefore, scraping each entry's page for number of references was done separately, as the order was sure to vary. Spider ["references"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/references.py) scraped for number of references.  The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_reference_counts in [wp_reference_counts.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_reference_counts_2.db), with the results falling ~13,000 rows short of the 133,769 rows captured by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py).
- Finally, the [June, 2022](https://en.wikipedia.org/wiki/Deaths_in_2022#June) page was scraped by Spider ["June_2022"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/June_2022.py), successfully capturing all of the previous fields, including number of references.  Note that the number of pages was some order of magnitude smaller.
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_deaths_June_2022 in [wp_deaths_June_2022.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_June_2022.db), resulting in 145 rows from the first part of June, 2022.

### 6/10/2022
The remaining data collection steps are outlined in this notebook:
1. [Reading, Sampling, and Checking Data Shape](#step1)
    - SQLite tables wp_deaths_94_to_22, wp_reference_counts, and wp_June_2022 were read in as pandas dataframes.
2. [Combining Dataframes](#step2)
    - Dataframes for wp_deaths_94_to_22 and wp_reference_counts were combined using `link` as the unique identifier.
    - Dataframe for wp_deaths_June_2022 was added.
3. [Duplicate Rows](#step3)
    - 9 rows of duplicate entries were dropped.
4. [Missing Values](#step4)
    - 5 rows missing essential data were dropped.
    - 6 entries had missing `name` and `num_references`, but contained the name in the `info` feature, to be extracted later during data cleaning.  As these entries had no associated page, their `num_references` was set equal to to 0.
5. [Missing `num_references` Values](#step5)
    - A single modification was made to the original XPath for the original "by_year" Spider, to match a variation on the pages for links with missing `num_references`.  
    - Those pages were then were rescraped iteratively by Spiders ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py), ["refs3"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs3.py), ["refs4"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs4.py), and ["refs5"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs5.py).
    - The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote their respective results to SQLite tables refs2 in [refs2.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs2.db), refs3 in [refs3.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs3.db), refs4 in [refs4.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs4.db), and refs5 in [refs5.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs5.db).
    - During each iteration, the additional `num_references` were added to the main dataset and remaining links with missing `num_references` values were identified.
    - After the final iteration, 92 entries remained with missing `num_references` that were all missing an associated page, so `num_references` was set equal to to 0.  These entries contained all of the other relevant features, so were preserved.
    - The resultant raw dataset has 133,900 rows and 6 columns.

\*  The current version of [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py) reflects its use by the most recent project spider crawled, as it is reused for all spiders within the [Scrapy project folder](https://github.com/teresahanak/wikipedia-notable-deaths/tree/main/wikipedia_notable_deaths).

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

<a id='step1'></a>
### Reading, Sampling, and Checking Data Shape

### January 1994 through May 2022 Data (without reference counts)

In [2]:
# Reading the wp_deaths_94_to_22 dataset from sql db and table
conn = sql.connect("wp_deaths_94_to_22.db")
raw_94_to_22 = pd.read_sql("SELECT * FROM wp_deaths_94_to_22", conn)

# Making a working copy
df_94_to_22 = raw_94_to_22.copy()

# Checking the shape
print(f"There are {df_94_to_22.shape[0]} rows and {df_94_to_22.shape[1]} columns.")

# Checking first 2 rows of the data
df_94_to_22.head(2)

There are 133769 rows and 5 columns.


,month_year,day,name,info,link
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer)
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df_94_to_22.tail(2)

,month_year,day,name,info,link
133767,May 2022,31,Dave Smith,", 72, American sound engineer, founder of Sequential.",https://en.wikipedia.org/wiki/Dave_Smith_(engineer)
133768,May 2022,31,Wang Zherong,", 86, Chinese tank designer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Wang_Zherong


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df_94_to_22.sample(5)

,month_year,day,name,info,link
3978,July 1995,29,Canray Fontenot,", 72, American Creole fiddle player.",https://en.wikipedia.org/wiki/Canray_Fontenot
38146,August 2008,31,Edwin O. Guthman,", 89, American Pulitzer Prize–winning journalist, amyloidosis.",https://en.wikipedia.org/wiki/Edwin_O._Guthman
115247,August 2020,11,Gordon J. Brand,", 65, English golfer.",https://en.wikipedia.org/wiki/Gordon_J._Brand
35180,November 2007,19,Laulu Fetauimalemau Mata'afa,", 79, Samoan educator, community worker, diplomat and former Member of Parliament.",https://en.wikipedia.org/wiki/Laulu_Fetauimalemau_Mata%27afa
57555,September 2012,8,Elizabeth Wood-Ellem,", 81, Tongan historian.",https://en.wikipedia.org/wiki/Elizabeth_Wood-Ellem


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,769 rows and 5 columns in the data from January, 1994 through May, 2022.
- The number of references was scraped separately.

### January 1994 through May 2022 Reference Count Data

In [5]:
# Reading the wp_reference_counts_2 dataset from sql db and table
conn = sql.connect("wp_reference_counts_2.db")
raw_reference_counts = pd.read_sql("SELECT * FROM wp_reference_counts_2", conn)

# Making a working copy
df_reference_counts = raw_reference_counts.copy()

# Checking the shape
print(
    f"There are {df_reference_counts.shape[0]} rows and {df_reference_counts.shape[1]} columns."
)

# Checking first 2 rows of the data
df_reference_counts.head(2)

There are 120368 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Lys_Gauty,5
1,https://en.wikipedia.org/wiki/William_Chappell_(dancer),21


<IPython.core.display.Javascript object>

In [6]:
# Checking last 2 rows of the data
df_reference_counts.tail(2)

,link,num_references
120366,https://en.wikipedia.org/wiki/Shirley_Thomas_(USC_professor),6
120367,https://en.wikipedia.org/wiki/James_Doohan,52


<IPython.core.display.Javascript object>

In [7]:
# Checking a sample of the data
df_reference_counts.sample(5)

,link,num_references
72252,https://en.wikipedia.org/wiki/A._Dean_Byrd,22
27205,https://en.wikipedia.org/wiki/Igor_Yefimov,3
45251,https://en.wikipedia.org/wiki/Dorothy_Eck,8
25279,https://en.wikipedia.org/wiki/Maria_Fyfe,8
100378,https://en.wikipedia.org/wiki/George_Reedy,1


<IPython.core.display.Javascript object>

#### Observations:
- Here, we see that there are ~13,000 fewer rows for the reference data, indicating some pages were not successfully scraped to obtain the number of references for the individual.
- After combining the three dataframes, we can examine those pages and reattempt scraping them, in order to obtain the missing information.

### June 2022 Data

In [8]:
# Reading the wp_deaths_June_2022 dataset from sql db and table
conn = sql.connect("wp_deaths_June_2022.db")
raw_June_2022 = pd.read_sql("SELECT * FROM wp_deaths_June_2022", conn)

# Making a working copy
df_June_2022 = raw_June_2022.copy()

# Checking the shape
print(f"There are {df_June_2022.shape[0]} rows and {df_June_2022.shape[1]} columns.")

# Checking first 2 rows of the data
df_June_2022.head(2)

There are 145 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,June 2022,8,Mladen Frančić,", 67, Croatian football player and manager (Vrbovec, Podravina, Al-Watani Club).",https://en.wikipedia.org/wiki/Mladen_Fran%C4%8Di%C4%87,1
1,June 2022,6,Valery Ryumin,", 82, Russian cosmonaut (Soyuz 25, Soyuz 32, Soyuz 35).",https://en.wikipedia.org/wiki/Valery_Ryumin,2


<IPython.core.display.Javascript object>

#### Observations:
- The June, 2022 data does not follow the same row order as the previous dataframe, which was in order of day of the month. 
- For continuity, before concatinating the two dataframes, we will sort June, 2022 by day.

In [9]:
# Sorting by day
df_June_2022.sort_values(by="day", inplace=True)

# Re-checking first 2 rows of the data
df_June_2022.head(2)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
20,June 2022,1,István Szőke,", 75, Hungarian footballer (Ferencváros, national team), stroke.",https://en.wikipedia.org/wiki/Istv%C3%A1n_Sz%C5%91ke,2


<IPython.core.display.Javascript object>

In [10]:
# Checking last 2 rows of the data
df_June_2022.tail(2)

,month_year,day,name,info,link,num_references
8,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
5,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [11]:
# Checking a sample of the data
df_June_2022.sample(5)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
7,June 2022,6,A. L. Mestel,", 95, American pediatric surgeon and visual artist.",https://en.wikipedia.org/wiki/A._L._Mestel,4
85,June 2022,4,Hu Fa-kuang,", 98, Hong Kong businessman and politician, member of the Legislative Council (1979–1988).",https://en.wikipedia.org/wiki/Hu_Fa-kuang,3
55,June 2022,2,Hal Bynum,", 87, American songwriter (""Lucille"", ""Chains"", ""Papa Was a Good Man""), complications from a stroke and Alzheimer's disease.",https://en.wikipedia.org/wiki/Hal_Bynum,3
83,June 2022,4,Beryl J. Levine,", 86, Canadian-born American judge, justice on the North Dakota Supreme Court (1985–1996).",https://en.wikipedia.org/wiki/Beryl_J._Levine,1


<IPython.core.display.Javascript object>

#### Observations:
- Now, we are ready to combine the three dataframes.

<a id='step2'></a>
## Combining Dataframes

### Adding Number of References to 1994 through May 2022 Data

In [12]:
# Adding num_references column to 1994 through May 2022 data
df_combined = pd.merge(df_94_to_22, df_reference_counts, how="left", on="link")

# Checking first 2 rows of the data
df_combined.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

### Adding June 2022 Data

In [13]:
# Adding Juned 2022 data
df_combined = pd.concat([df_combined, df_June_2022], ignore_index=True)

# Making a working copy
df = df_combined.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133914 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [14]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133912,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133913,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [15]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
1676,September 1994,20,Jule Styne,", 88, English-American songwriter and composer.",https://en.wikipedia.org/wiki/Jule_Styne,12
31020,June 2006,29,Pierre Rinfret,", 82, Canadian-born economist and Republican candidate for Governor of New York in 1990.",https://en.wikipedia.org/wiki/Pierre_Rinfret,0
93229,November 2017,20,Jean Hearn,", 96, Australian politician, Senator (1980–1985).",https://en.wikipedia.org/wiki/Jean_Hearn,3
66606,January 2014,9,Cliff Carpenter,", 98, American actor (, , ).",https://en.wikipedia.org/wiki/Cliff_Carpenter,6
101876,January 2019,27,Anant Prasad Singh,", 92, Indian politician, MLA (1969–1977).",https://en.wikipedia.org/wiki/Anant_Prasad_Singh,3


<IPython.core.display.Javascript object>

#### Confirming Correct Number of Resultant Entries

In [16]:
# Confirming correct number of total rows
df_94_to_22.shape[0] + df_June_2022.shape[0]

del df_94_to_22, df_reference_counts, df_June_2022

<IPython.core.display.Javascript object>

#### Observations:
- We have successfully combined the three dataframes.
- Now, we can check for data types, duplicates, and missing values.

<a id='step3'></a>
## Checking Data Types, Duplicates, and Null Values

### Data types

In [17]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133914 entries, 0 to 133913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133914 non-null  object
 1   day             133914 non-null  object
 2   name            133903 non-null  object
 3   info            133914 non-null  object
 4   link            133914 non-null  object
 5   num_references  120637 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are 6 columns, all of object type.
- `name` and `num_references` both have missing values.
- The data is in a very raw format and there are other columns that have combined information that will need to be extracted.
- For now, we will leave all as object type.

<a id='step3'></a>
### Duplicate Rows

In [18]:
# Checking duplicate rows
df.duplicated().sum()

9

<IPython.core.display.Javascript object>

#### Observations:
- There are 9 duplicate rows that we will drop now.

In [19]:
# Drop duplicate rows
df.drop_duplicates(inplace=True, ignore_index=True)

# Re-check shape
df.shape

(133905, 6)

<IPython.core.display.Javascript object>

<a id='step4'></a>
### Missing Values

In [20]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000
day               0.000
name              0.008
info              0.000
link              0.000
num_references   11.007
dtype: float64

<IPython.core.display.Javascript object>

In [21]:
# Checking number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    120628
1     13266
2        11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The number of rows missing only 1 value appears consistent with our anticipated missing `num_references`.
- There are 11 rows that are each missing 2 values.  Let us take a closer look at these rows.

In [22]:
# Checking the rows that are missing values for 2 columns
missing_2 = df[df.isnull().sum(axis=1) == 2]
missing_2

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,NaN
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,NaN
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,NaN
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,NaN
35097,November 2007,11,None,,https://en.wikipedia.orgNone,NaN
41075,May 2009,18,None,Either killed in a missile attack or shot:\n,https://en.wikipedia.orgNone,NaN
64771,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,NaN
76024,April 2015,29,None,Notable convicted drug traffickers executed by Indonesian firing squad:\n,https://en.wikipedia.orgNone,NaN
105871,August 2019,2,None,"Japanese convicted murderers, executed by hanging.\n",https://en.wikipedia.orgNone,NaN
106617,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can see that multiple rows are missing `name`, but have the name in `info`, so we can extract it later.  
- The missing link itself is not of concern as it serves only as a means by which to retrieve the `num_references` value.
- As there is no associated link for the individual, we can safely replace the NaN `num_references` values for rows with extractable names with 0.
- We can proceed with removing the rows that lack an extractable name, as they also lack other information necessary for the analysis.

In [23]:
# List of rows to keep
keep_rows = [18937, 24985, 27458, 34077, 64771, 106617]

# For loop to replace num_references NaNs with 0 for rows with extractable names
for row in keep_rows:
    df.loc[row, "num_references"] = 0

# List of rows to remove
remove_rows = [index for index in missing_2.index if index not in keep_rows]
del missing_2

# Dropping rows
df.drop(remove_rows, inplace=True)

# Re-checking shape
df.shape

(133900, 6)

<IPython.core.display.Javascript object>

In [24]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133900 entries, 0 to 133904
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  120634 non-null  object
dtypes: object(6)
memory usage: 7.2+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are now only 6 rows with missing `name`, corresponding to the names we identified in `info`, that we will extract later.
- The remaining missing values are all for `num_references`, so we can proceed to make another attempt at scraping this information.
- Let us check a sample of these rows.

In [25]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
46821,August 2010,20,Jack Horkheimer,", 72, American public television host (Miami Planetarium), executive director of , respiratory ailment.",https://en.wikipedia.org/wiki/Jack_Horkheimer,NaN
34223,July 2007,28,Sal Mosca,", 80, American jazz pianist and educator.",https://en.wikipedia.org/wiki/Sal_Mosca,NaN
81686,March 2016,9,Galagama Sri Aththadassi Thera,", 94, Sri Lankan Buddhist monk, Mahanayaka of the Asgiriya Chapter of Siyam Nikaya (since 2015).",https://en.wikipedia.org/wiki/Galagama_Sri_Aththadassi_Thera,NaN
29552,December 2005,11,Hayim Tadmor,", 82, Israeli Assyriologist and professor.",https://en.wikipedia.org/wiki/Hayim_Tadmor,NaN
36065,February 2008,9,Manuel F. Alsina Capo,", 98, Spanish-Puerto Rican urologist and surgeon.",https://en.wikipedia.org/wiki/Manuel_F._Alsina_Capo,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links reveals that the pages contain references.  
- Therefore, they either have a variation in the XPath followed for scraping, or Scrapy had an issue with following their links.
- We will export a dataframe of the links to the pages that need to be re-scraped for `num_references`.

In [26]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df = df[df["num_references"].isna()]["link"]
rescrape_df.to_csv("rescrape_df.csv", index=False)

del rescrape_df

<IPython.core.display.Javascript object>

<a id='step5'></a>
## Missing `num_references` Values

### First Re-scrape with Spider ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py)

#### Observations:
- A second iteration of scraping individual pages for number of references reveals variation in the XPath for those pages.
- The new spider, ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py), used an updated XPath, with the identified variation.
- As before, we will import and merge the data, examine rows still missing `num_references` and re-scrape them, adjusting the spider's XPath again, if needed.

In [27]:
# Reading the refs2 dataset from sql db and table
conn = sql.connect("refs2.db")
raw_refs2 = pd.read_sql("SELECT * FROM refs2", conn)

# Making a working copy
df_refs2 = raw_refs2.copy()

# Checking the shape
print(f"There are {df_refs2.shape[0]} rows and {df_refs2.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs2.head(2)

There are 7365 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Charlotte_Benkner,63
1,https://en.wikipedia.org/wiki/Eugene_Record,11


<IPython.core.display.Javascript object>

In [28]:
# Checking last 2 rows of the data
df_refs2.tail(2)

,link,num_references
7363,https://en.wikipedia.org/wiki/Gunnar_Utterberg,3
7364,https://en.wikipedia.org/wiki/Bill_Sudakis,19


<IPython.core.display.Javascript object>

In [29]:
# Checking a sample of the data
df_refs2.sample(5)

,link,num_references
5885,https://en.wikipedia.org/wiki/Jerry_Williams_(singer),24
2056,https://en.wikipedia.org/wiki/Wendall_Anschutz,4
3919,https://en.wikipedia.org/wiki/Mimi_Cazort,3
2011,https://en.wikipedia.org/wiki/Peter_Chen_Bolu,2
5667,https://en.wikipedia.org/wiki/Arthur_S._Abramson,8


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 7365 of the missing values.

#### Adding Missing References to Dataframe

In [30]:
# Adding new num_references column to data
df = pd.merge(df, df_refs2, how="left", on="link")
del df_refs2

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
86697,December 2016,13,Barrelhouse Chuck,", 58, American blues musician, prostate cancer.",https://en.wikipedia.org/wiki/Barrelhouse_Chuck,11,NaN
123945,May 2021,28,Esther Brown,", 67, British pensioner, beaten.",https://en.wikipedia.org/wiki/Esther_Brown_(pensioner),12,NaN
390,March 1994,1,Manmohan Desai,", 57, Indian film producer and director.",https://en.wikipedia.org/wiki/Manmohan_Desai,9,NaN
59572,December 2012,28,Fred Rehm,", 91, American basketball player.",https://en.wikipedia.org/wiki/Fred_Rehm,3,NaN
1833,October 1994,15,Jean Dasté,", 90, Frenck actor and theatre director.",https://en.wikipedia.org/wiki/Jean_Dast%C3%A9,1,NaN


<IPython.core.display.Javascript object>

In [31]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
91195,July 2017,28,Maurice Filion,", 85, Canadian ice hockey executive and coach (Quebec Nordiques).",https://en.wikipedia.org/wiki/Maurice_Filion,NaN,NaN
41365,June 2009,17,"Ralf Dahrendorf, Baron Dahrendorf",", 80, German-born British sociologist and politician, cancer.",https://en.wikipedia.org/wiki/Ralf_Dahrendorf,39,NaN
75460,April 2015,1,Misao Okawa,", 117, Japanese supercentenarian, world's oldest living person.",https://en.wikipedia.org/wiki/Misao_Okawa,42,NaN
6130,May 1996,21,Vladimir Belyakov,", 78, Soviet gymnast.",https://en.wikipedia.org/wiki/Vladimir_Belyakov,0,NaN
67036,January 2014,30,William Motzing,", 77, American-born Australian composer, conductor and arranger.",https://en.wikipedia.org/wiki/William_Motzing,9,NaN


<IPython.core.display.Javascript object>

In [32]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [33]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    5895
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have nearly 6000 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [34]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
41958,August 2009,9,Jasmine You,", 30, Japanese bassist (Versailles).",https://en.wikipedia.org/wiki/Jasmine_You,NaN
74864,March 2015,1,Matthew Young,", 70, British civil servant and executive (Panini Group).",https://en.wikipedia.org/wiki/Matthew_Young_(civil_servant),NaN
118081,December 2020,5,Anusuya Prasad Maikhuri,", 59, Indian politician, Uttarakhand MLA (2002–2007, 2012–2017), complications from COVID-19.",https://en.wikipedia.org/wiki/Anusuya_Prasad_Maikhuri,NaN
113058,May 2020,22,Zara Abid,", 28, Pakistani model and actress (plane crash), .",https://en.wikipedia.org/wiki/Zara_Abid,NaN
80672,January 2016,20,Eva Schorr,", 88, German painter and composer.",https://en.wikipedia.org/wiki/Eva_Schorr,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [35]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_2nd = df[df["num_references"].isna()]["link"]
rescrape_df_2nd.to_csv("rescrape_df_2nd.csv", index=False)
del rescrape_df_2nd

<IPython.core.display.Javascript object>

### Second Re-scrape with Spider ["refs3"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs3.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [36]:
# Reading the refs3 dataset from sql db and table
conn = sql.connect("refs3.db")
raw_refs3 = pd.read_sql("SELECT * FROM refs3", conn)

# Making a working copy
df_refs3 = raw_refs3.copy()

# Checking the shape
print(f"There are {df_refs3.shape[0]} rows and {df_refs3.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs3.head(2)

There are 3633 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Grace_Thaxton,63
1,"https://en.wikipedia.org/wiki/Christopher_Prout,_Baron_Kingsland",6


<IPython.core.display.Javascript object>

In [37]:
# Checking last 2 rows of the data
df_refs3.tail(2)

,link,num_references
3631,https://en.wikipedia.org/wiki/Concepci%C3%B3n_Ram%C3%ADrez,8
3632,https://en.wikipedia.org/wiki/Mary_Mahoney_(physician),6


<IPython.core.display.Javascript object>

In [38]:
# Checking a sample of the data
df_refs3.sample(5)

,link,num_references
3174,https://en.wikipedia.org/wiki/Hogan_Sheffer,7
2964,https://en.wikipedia.org/wiki/George_Brown_(athlete),25
3466,https://en.wikipedia.org/wiki/Alice_Recoque,10
978,https://en.wikipedia.org/wiki/Nolan_Luhn,2
675,https://en.wikipedia.org/wiki/Wu_Jieping,3


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 3633 of the missing values.

#### Adding Missing References to Dataframe

In [39]:
# Adding new num_references column to data
df = pd.merge(df, df_refs3, how="left", on="link")
del df_refs3

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
126469,August 2021,29,Alex Gallacher,", 67, Scottish-born Australian politician, senator (since 2011), lung cancer.",https://en.wikipedia.org/wiki/Alex_Gallacher,10,NaN
49880,April 2011,23,Ed Austin,", 84, American attorney and politician, Mayor of Jacksonville, Florida (1991–1995).",https://en.wikipedia.org/wiki/Ed_Austin,9,NaN
95773,March 2018,18,Cloria Brown,", 75, American politician, member of the Missouri House of Representatives (2011–2013, since 2015), cancer.",https://en.wikipedia.org/wiki/Cloria_Brown,8,NaN
116504,October 2020,4,Clark Middleton,", 63, American actor (, , ), West Nile virus.",https://en.wikipedia.org/wiki/Clark_Middleton,5,NaN
108066,November 2019,25,Nobuaki Kobayashi,", 77, Japanese three-cushion billiards player, two-time world champion.",https://en.wikipedia.org/wiki/Nobuaki_Kobayashi,5,NaN


<IPython.core.display.Javascript object>

In [40]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
53440,January 2012,13,Oscar Valentín Leal Caal,", 41, Guatemalan politician, Congressman (since 2008), shot.",https://en.wikipedia.org/wiki/Oscar_Valent%C3%ADn_Leal_Caal,3,NaN
10347,December 1997,22,Clara Lee Tanner,", 92, American anthropologist and art historian.",https://en.wikipedia.org/wiki/Clara_Lee_Tanner,15,NaN
29070,October 2005,10,Aivaras Balzekas,", 23, Lithuanian tennis player, car accident.",https://en.wikipedia.org/wiki/Aivaras_Bal%C5%BEekas,3,NaN
127594,October 2021,10,Abdul Qadeer Khan,", 85, Pakistani nuclear physicist, complications from COVID-19.",https://en.wikipedia.org/wiki/Abdul_Qadeer_Khan,81,NaN
19914,December 2001,19,Hans Warren,", 80, Dutch writer, liver problems.",https://en.wikipedia.org/wiki/Hans_Warren,4,NaN


<IPython.core.display.Javascript object>

In [41]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [42]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    2260
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have 2260 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [43]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
70992,August 2014,16,Mario Oriani-Ambrosini,", 53, Italian-born South African politician, MP (since 2009), lung cancer.",https://en.wikipedia.org/wiki/Mario_Oriani-Ambrosini,NaN
75715,April 2015,13,Ronnie Carroll,", 80, Northern Irish singer and political candidate.",https://en.wikipedia.org/wiki/Ronnie_Carroll,NaN
85086,September 2016,12,Tor Brustad,", 89, Norwegian biophysicist.",https://en.wikipedia.org/wiki/Tor_Brustad,NaN
108470,December 2019,14,Billie Rattigan,", 87, Irish Gaelic footballer (Dunshaughlin, Meath).",https://en.wikipedia.org/wiki/Billie_Rattigan,NaN
73945,January 2015,14,Nélida Romero,", 88, Argentine actress.",https://en.wikipedia.org/wiki/N%C3%A9lida_Romero,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [44]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_3rd = df[df["num_references"].isna()]["link"]
rescrape_df_3rd.to_csv("rescrape_df_3rd.csv", index=False)
del rescrape_df_3rd

<IPython.core.display.Javascript object>

### Third Re-scrape with Spider ["refs4"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs4.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [45]:
# Reading the refs4 dataset from sql db and table
conn = sql.connect("refs4.db")
raw_refs4 = pd.read_sql("SELECT * FROM refs4", conn)

# Making a working copy
df_refs4 = raw_refs4.copy()

# Checking the shape
print(f"There are {df_refs4.shape[0]} rows and {df_refs4.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs4.head(2)

There are 1960 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_French_supercentenarians,157
1,https://en.wikipedia.org/wiki/Bob_Kahler,3


<IPython.core.display.Javascript object>

In [46]:
# Checking last 2 rows of the data
df_refs4.tail(2)

,link,num_references
1958,https://en.wikipedia.org/wiki/Jorge_Sampaio,73
1959,https://en.wikipedia.org/wiki/Irma_Kalish,14


<IPython.core.display.Javascript object>

In [47]:
# Checking a sample of the data
df_refs4.sample(5)

,link,num_references
1737,https://en.wikipedia.org/wiki/M%C3%A1rio_Chermont,3
1178,https://en.wikipedia.org/wiki/Walter_Babington_Thomas,15
1233,https://en.wikipedia.org/wiki/Joe_Ellis_Brown,4
375,https://en.wikipedia.org/wiki/Isabelle_Collin_Dufresne,21
1496,https://en.wikipedia.org/wiki/Mark_Farrell_(tennis),5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 1960 of the missing values.

#### Adding Missing References to Dataframe

In [48]:
# Adding new num_references column to data
df = pd.merge(df, df_refs4, how="left", on="link")
del df_refs4

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
9319,August 1997,10,Carlton Moss,", 88, American screenwriter, actor and film director.",https://en.wikipedia.org/wiki/Carlton_Moss,7,NaN
62046,April 2013,27,Maurice Gransart,", 82, French footballer (Olympique de Marseille).",https://en.wikipedia.org/wiki/Maurice_Gransart,NaN,2
104585,June 2019,2,Lowell North,", 89, American sailor, five-time world champion, Olympic champion (1968).",https://en.wikipedia.org/wiki/Lowell_North,10,NaN
47594,October 2010,23,George Cain,", 66, American author, kidney failure.",https://en.wikipedia.org/wiki/George_Cain,2,NaN
101639,January 2019,18,John Krogh,", 80, Norwegian footballer (Rosenborg, national team).",https://en.wikipedia.org/wiki/John_Krogh,2,NaN


<IPython.core.display.Javascript object>

In [49]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
76100,May 2015,4,William Bast,", 84, American screenwriter and author, Alzheimer's disease.",https://en.wikipedia.org/wiki/William_Bast,10,NaN
87621,January 2017,25,Mary Tyler Moore,", 80, American actress (Emmy, , ), 7-time winner, cardiopulmonary arrest.",https://en.wikipedia.org/wiki/Mary_Tyler_Moore,115,NaN
29493,December 2005,3,Atsuko Tanaka,", 74, Japanese avant-garde artist.",https://en.wikipedia.org/wiki/Atsuko_Tanaka_(artist),24,NaN
81574,March 2016,4,Yuri Kuznetsov,", 83, Azerbaijani Soviet football player and coach (Neftchi).","https://en.wikipedia.org/wiki/Yuri_Kuznetsov_(footballer,_born_1931)",2,NaN
113243,May 2020,29,Hank Mason,", 88, American baseball player (Philadelphia Phillies).",https://en.wikipedia.org/wiki/Hank_Mason,5,NaN


<IPython.core.display.Javascript object>

In [50]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [51]:
# Checking remaining missing values
df.isna().sum()

month_year          0
day                 0
name                6
info                0
link                0
num_references    300
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 300 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [52]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
133740,May 2022,31,Aripah Damanhuri,", 72, Malaysian actress.",https://en.wikipedia.org/w/index.php?title=Aripah_Damanhuri&action=edit&redlink=1,NaN
126378,August 2021,25,Ted Dexter,", 86, English cricketer (Sussex, national team).",https://en.wikipedia.org/wiki/Ted_Dexter,NaN
125244,July 2021,17,Aleksandr Starovoitov,", 80, Russian security officer and academic, director of FAPSI (1991–1998).",https://en.wikipedia.org/wiki/Aleksandr_Starovoitov,NaN
133502,May 2022,19,Jerzy Zass,", 84, Polish actor (, , ).",https://en.wikipedia.org/w/index.php?title=Jerzy_Zass&action=edit&redlink=1,NaN
120179,February 2021,3,Ali Ansarian,", 43, Iranian footballer (Persepolis, Shahrdari Tabriz, national team), COVID-19.",https://en.wikipedia.org/wiki/Ali_Ansarian,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [53]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_4th = df[df["num_references"].isna()]["link"]
rescrape_df_4th.to_csv("rescrape_df_4th.csv", index=False)
del rescrape_df_4th

<IPython.core.display.Javascript object>

### Fourth Re-scrape with Spider ["refs5"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs5.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [54]:
# Reading the refs5 dataset from sql db and table
conn = sql.connect("refs5.db")
raw_refs5 = pd.read_sql("SELECT * FROM refs5", conn)

# Making a working copy
df_refs5 = raw_refs5.copy()

# Checking the shape
print(f"There are {df_refs5.shape[0]} rows and {df_refs5.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs5.head(2)

There are 208 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Mohamed_Haytham_Khayat,5
1,https://en.wikipedia.org/wiki/Sid_McCray,6


<IPython.core.display.Javascript object>

In [55]:
# Checking last 2 rows of the data
df_refs5.tail(2)

,link,num_references
206,https://en.wikipedia.org/wiki/Anna_Cataldi,9
207,https://en.wikipedia.org/wiki/Allan_Egolf,4


<IPython.core.display.Javascript object>

In [56]:
# Checking a sample of the data
df_refs5.sample(5)

,link,num_references
188,https://en.wikipedia.org/wiki/Kenny_Malone,5
97,https://en.wikipedia.org/wiki/Guillermo_Galeote,2
130,https://en.wikipedia.org/wiki/Fausto_Gresini,3
94,https://en.wikipedia.org/wiki/Godfrey_Hodgson,8
147,https://en.wikipedia.org/wiki/Margaret_Maron,12


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 208 of the missing values.

#### Adding Missing References to Dataframe

In [57]:
# Adding new num_references column to data
df = pd.merge(df, df_refs5, how="left", on="link")
del df_refs5

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
34208,July 2007,26,Shambo,", 6, British Hindu sacred bull, lethal injection due to bovine tuberculosis.",https://en.wikipedia.org/wiki/Shambo,24,NaN
20855,April 2002,24,Gloria Escoffery,", 78, Jamaican artist, poet, teacher, art critic and journalist.",https://en.wikipedia.org/wiki/Gloria_Escoffery,3,NaN
54153,February 2012,19,Eric Harris,", 56, American football player (Toronto Argonauts, Kansas City Chiefs, Los Angeles Rams), heart attack.",https://en.wikipedia.org/wiki/Eric_Harris_(gridiron_football),4,NaN
125200,July 2021,16,Anthony Adams,", 80, Australian-American optometrist.",https://en.wikipedia.org/wiki/Anthony_Adams_(optometrist),4,NaN
132652,April 2022,9,Michel Delebarre,", 75, French politician, mayor of Dunkirk (1989–2014) and senator (2011–2017).",https://en.wikipedia.org/wiki/Michel_Delebarre,4,NaN


<IPython.core.display.Javascript object>

In [58]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
94275,January 2018,10,Damdinjavyn Bandi,", 75, Mongolian Olympic boxer (1972, 1976).",https://en.wikipedia.org/wiki/Damdinjavyn_Bandi,2,NaN
31566,September 2006,5,Hilary Mason,", 89, British character actress.",https://en.wikipedia.org/wiki/Hilary_Mason,9,NaN
34253,August 2007,1,Robert Hughes,", 95, Australian composer.",https://en.wikipedia.org/wiki/Robert_Hughes_(composer),1,NaN
41971,August 2009,10,Zarema Sadulayeva,", 33, Russian activist, head of children's aid organization in Chechnya, shot.",https://en.wikipedia.org/wiki/Zarema_Sadulayeva,4,NaN
37156,May 2008,22,Charlie Booth,", 104, Australian athlete, inventor of the starting block.",https://en.wikipedia.org/wiki/Charlie_Booth,4,NaN


<IPython.core.display.Javascript object>

In [59]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [60]:
# Checking remaining missing values
df.isna().sum()

month_year         0
day                0
name               6
info               0
link               0
num_references    92
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 92 remaining missing values for `num_references`.
- Let us examine the remaining rows with missing values more closely.

In [61]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()]

,month_year,day,name,info,link,num_references
12476,October 1998,29,Sjoerdtsje Faber,", 83, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sjoerdje_Faber&action=edit&redlink=1,NaN
15859,May 2000,10,Bill Foster,", 68, American entertainer.",https://en.wikipedia.org/w/index.php?title=Bill_Foster_(performer)&action=edit&redlink=1,NaN
18561,June 2001,20,Demetreus Nix,"Douglas Scott, 20, High-school student murdered by .",https://en.wikipedia.org/w/index.php?title=Demetreus_Nix&action=edit&redlink=1,NaN
19905,December 2001,18,Sietske Pasveer,", 86, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sietske_Pasveer&action=edit&redlink=1,NaN
22942,March 2003,3,Gilbert Wheeler Beebe,", 90, American epidemiologist and statistician, conducted ground-breaking radiation exposure studies.",https://en.wikipedia.org/w/index.php?title=Gilbert_Wheeler_Beebe&action=edit&redlink=1,NaN
23291,April 2003,27,Charles A. Marvin,", 73, American district attorney and judge.",https://en.wikipedia.org/w/index.php?title=Charles_A._Marvin&action=edit&redlink=1,NaN
24346,October 2003,15,Ray Kuhlman,", 84, American pilot and businessman.",https://en.wikipedia.org/w/index.php?title=Ray_Kuhlman&action=edit&redlink=1,NaN
33653,May 2007,20,Moses Siregar,", 96, Maritime Chef and American World War 2 Veteran",https://en.wikipedia.org/w/index.php?title=Moses_Siregar&action=edit&redlink=1,NaN
35963,January 2008,31,Michael A. Dions,", 90, American Olympic swimmer, natural causes",https://en.wikipedia.org/w/index.php?title=Michael_A._Dions&action=edit&redlink=1,NaN
36198,February 2008,19,Samuel Champkin,", 28, American singer in Metal band Tech Giants, car crash.",https://en.wikipedia.org/w/index.php?title=Samuel_Champkin&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- None of the remaining links direct to a personal page for the individual, as such a page does not yet exist.
- As such, as the rows contain the other elements necessary for analysis, it is safe to replace the missing `num_references` values with 0, for these rows.

In [62]:
# Fill remaining missing values for num_references with 0
df["num_references"].fillna(0, inplace=True)

# Recheck missing values
df.isna().sum()

month_year        0
day               0
name              6
info              0
link              0
num_references    0
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- All of the missing values for `num_references` have been addressed.
- The 6 remaining missing values for `name` will be fixed during data cleaning.
- We now have our complete raw dataset, which we will also export to a SQLite database.
- Then it is time to start cleaning the data.

### Exporting Dataset to SQLite Database [wp_life_expect_raw_complete.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_raw_complete.db)

In [63]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_life_expect_raw_complete.db")
df.to_sql("wp_life_expect_raw_complete", conn, index=False)

133900

<IPython.core.display.Javascript object>

# [Proceed to Notebook 2 of  4:  Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean_thanak_2022_06_13.ipynb)